In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, sys
sys.path.insert(0, '../scripts')
from fc_helpers import make_corrfig, linearize, get_flat_inds_for_net, triangularizeweights
from permute_helpers import prep_data, run_randomforest_permutation_loocv
from sklearn.model_selection import  LeaveOneOut
from sklearn.ensemble import RandomForestClassifier



In [2]:
X_scale, y = prep_data()

In [4]:
netlist = ['Auditory','CingOperc','CingPar','Default','DorsalAtt','FrontoPar','None', 'RetroTemp','Salience', 
           'SMhand','SMmouth','VentralAtt','Visual','Subcort']   


In [31]:
def create_null(n_sample, pop_size = 62128): return random.sample(range(0,pop_size), n_sample)

In [32]:
create_null(2)

[36230, 17896]

In [48]:
def run_randomforest_permutation_nulls_loocv(X_mat, y, n_nulls = 15, estimators = 500, permutations = 50):
    

    cv = LeaveOneOut()
    scores = []
    netlist = ['Auditory','CingOperc','CingPar','Default','DorsalAtt','FrontoPar','None', 'RetroTemp','Salience', 
               'SMhand','SMmouth','VentralAtt','Visual','Subcort']   
    fold = 0

    # Loop over the LOOCV splits indicies
    for train_ix, test_ix in cv.split(X_mat):
            
        # For each split, create the respective training and test set
        X_train, X_test = X_mat[train_ix, :], X_mat[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        # Train the model
        clf = RandomForestClassifier(n_estimators=estimators)

        clf.fit(X_train, y_train)
            
        for net in netlist:
            
            for null in range(n_nulls):
                
                network_inds_len = len(get_flat_inds_for_net(net)
                null_inds = create_null(network_inds_len)
            
                temp_test_sub = np.copy(X_test)

                for j in range(permutations):

                    # Permute the randomly selected connections individually 
                    for i in range(len(null_inds)):
                        randsamp = random.randint(0,X_mat.shape[0]-2)
                        temp_test_sub[0, null_inds[i]] = X_train[randsamp, null_inds[i]]

                    # Test the model using the permuted feature set
                    loo_score = clf.score(temp_test_sub, y_test)

                    # Keep track of the accuracy of the LOOCV with DMN permuted
                    scores.append(loo_score)     

        if fold%50 == 0:
            print(fold)
        fold = fold +1

    return scores        
